## Введение в `astropy.coordinates`

После освоения `astropy.units` с его представлением физических величин, можно приступить к освоению центрального понятия астрономических вычислений: **небесных опорных систем отсчета**.

`astropy.coordinates` предоставляет мощный инструментарий для взаимодействия и обработки астрономических координат и движений. Он позволяет астрономам задать положения в разных координатных системах, совершать переход из одних координат в другие, и осуществлять такие операции, как проецирование, подсчет углового расстояния и полных скоростей.

Этот модуль является ключевым для задач, которые включают в себя:
- Небесные координаты (RA/Dec, галактические, горизонтальные)
- Собственные движения и лучевые скорости
- Преобразования координат (галактические ↔ ICRS ↔ горизонтальные)
- Планирования наблюдения и отслеживания координат нужного объекта

`astropy.coordinates` полностью сочетается с `astropy.units`, так что все углы, расстояния и скорости имеют единицы измерения, что обеспечивает физическое и вычислительное согласованность. 

В этом разделе, мы сфокусируемся на основном функционале модуля `astropy.coordinates` — определении, интерпретации и преобразовании астрономических положений. Обработка этого функционала будет необходима для работы с астрономическими базами данных и небесными каталогами, в особенности с использованием `astroquery`.

### Работа с углами в `astropy.coordinates`

Угол является ключевой величиной для астрономов, поскольку те чаще всего работают со сферической симметрией. Поэтому обработка угловых величин с точностью и постоянством является важной задачей. В `astropy.coordinates` для создания и манипуляций с углами есть класс `Angle`.

`Angle`, по сути, является заточенным специально для углов классом `Quantity`. Он позволяет:

- Гибкое создание из градусов, радианов, часов, шестидесятиричных представлений;
- Легкий и высокоточный переход из одного представления в другое
- Стабильную арифметику (сложение, вычитание, учет полного оборота) и сравнение углов

Короче говоря, класс `Angle` дает сразу два преимущества: **силу единиц измерения astropy.units** и **выразительность в астрономических угловых величинах**.

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle

print(Angle(10.2345 * u.deg))         # задание через Quantity

# Способы задания углов с помощью строк, принимается почти любая общеупотребимая форма
a = Angle('10.2345d') # десятичные градусы, d - degrees

b = Angle('1:2:30.43 degrees')     # Шестидесятиричные градусы
c = Angle('1d2m3.4s') 

d = Angle('1 2 0.34 hours') # Шестидесятиричные часы
e = Angle('-1h2m3s')

# вывод через Jupyter по-умолчанию в шестидесятиричном виде, посмотрите на вывод b, c, d, e
a

In [ ]:
# можно задавать в виде юникодовских символов градусов-минут-секунд
alpha = Angle('1°2′3″')     
# а ещё к любому обозначению можно добавить "направление" N/S или E/W, положительными считаются N и E
beta = Angle('1d2m3.4sS')
gamma = Angle('-1h2m3sW') 

gamma

In [ ]:
print(Angle(['10.2345d', '-20d']))    
print(Angle(np.arange(1., 8.), unit=u.deg)) # создавать можно сразу же массив углов из списков или ndarray

In [ ]:
# атрибуты угла позволяют вывести его величину в разных форматах
print(f"{alpha.deg:.5f}° =")
print(f"{alpha.arcmin:.2f}´ =")
print(f"{alpha.arcsec:.0f}´´ =")
print(f"{alpha.radian:.6f} рад =")
print(f"{alpha.hour:.6f}h")
print("")
print("Кортеж из часов-минут-секунд:")
print(alpha.hms)
print(alpha.hms.h, alpha.hms.m, alpha.hms.s)
print("")
print("Кортеж из градусов-минут-секунд:")
print(alpha.dms)
print(alpha.dms.d, alpha.dms.m, alpha.dms.s)

### Математические действия с углами

Классы `Angle` поддерживают все стандартные математические функции, реализованные в стандартном Python и в NumPy.

In [ ]:
a1 = Angle("9d30m")
a2 = Angle("-2h15m30s")
print(f"Входные углы: a₁ = {a1:unicode}")
print(f"a₂ = {a2.to(u.deg):unicode}")
a3 = a1 + a2
a4 = a1 - a2
print(f"Сумма углов: {a3:unicode}")
print(f"Разность углов: {a4:unicode}")
# не нужно переживать за передачу именно радиан в тригонометрические функции - это делается автоматически
print(f"Синус 90°: {np.sin(Angle('90d'))}")

## Задание положения с помощью класса SkyCoord

Когда мы говорим про положение объекта в трехмерном пространстве, мы задаем не один угол, а два ("широту" и "долготу"). По сути, эта пара углов является величиной векторной, которая преобразуется по определенным правилам и ведет себя, как единая сущность.

Правила поведения этой сущности отражены в объекте класса `SkyCoord` из модуля `astropy.coordinates`. Создадим такой для работы с положением шарового скопления **NGC 2808**:

In [ ]:
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=138.0108*u.degree, dec= -64.8625 *u.degree, frame='icrs') # NGC 2808
print(coord)
print(type(coord))

Мы предоставили:
- **Прямое восхождение (RA)**: 138.0108 градусов 
- **Склонение (Dec)**: −64.8625 градусов  
- **Систему отсчета**: `'icrs'` (Международная Небесная Опорная Система Отсчета), стандартная небесная система, базирующаяся на внегалактических реперных точках

Явно задавая единицы измерения углов `u.degree`, мы обеспечиваем постоянство единиц и позволяем объекту `SkyCoord` правильно взаимодействовать с остальными инструментами в экосистеме `astropy`.

Получившийся объект класса `SkyCoord` включает в себя положение и информацию о системе отсчета, служа основой для целей преобразования координат, расстояний, собственных движений и запросов в базы данных.

В качестве входных координат мы можем как явно передать объект класса `Angle`, так и строку-генератор для него. 

In [ ]:
#astropy.coordinates может переварить любое представление углов
coord = SkyCoord(138.0108, -64.8625, frame='icrs', unit='deg') # обозначаем общую единицу
print(coord)
coord = SkyCoord('09h12m2.6s', '-64d51m45s', frame='icrs') # обозначаем единицы в строковом представлении
print(coord)
coord = SkyCoord('09 12 2.6 -64 51 45', unit=(u.hourangle, u.deg)) # обозначаем в шестидесятиричном представлении
print(coord)
coord = SkyCoord('09:12:2.6 -64:51:45', unit=(u.hourangle, u.deg)) # и ещё раз, но с другим разделителем
print(coord)

In [ ]:
# вывести координтаы также можно практически в любом общеупотребимом виде
print(coord.ra, coord.dec) # .ra и .dec - атрибуты класса SkyCoord
print(type(coord.ra))
print(type(coord.dec)) # имеют немного разный тип для широтных и долготных углов, но все это перегруженный Angle

In [ ]:
# обращение к конкретным численным значениям происходит так же, как для Angle
print(coord.ra.hour, coord.dec.deg)
print(type(coord.ra.hour)) # тип - float, без единицы измерения!

In [ ]:
print(coord.ra.hms)
print(coord.ra.hms.h, coord.ra.hms.m, coord.ra.hms.s) 
print('')
print(coord.dec.dms)
print('')
print(coord.ra.radian, coord.dec.radian)

In [ ]:
print(coord.to_string('decimal')) # различные представления сразу пары координат
print(coord.to_string('dms')) 
print(coord.to_string('hmsdms')) # удобно для передачи в каталоги

## Простой пример преобразования координат

Представим наш объект типа `SkyCoord` в иной координатной системе, например, в галактической (l, b). Напомним:

- **Галактическая долгота (`l`)** — измеряется в градусах, увеличивается в направлении вращения диска Галактики
- **Галактическая широта (`b`)** — измеряется в градусах, положительная над галактической плоскостью

Переход из ICRS в галактические координаты осуществляется встроенной матрицей перехода, с общепринятыми началом отсчета (центром Галактики) и галактическим полюсом.

In [ ]:
print(coord.galactic) # автоперевод в галактические широту и долготу

## Поиск координат объекта по имени

Метод `SkyCoord.from_name(name)` позволяет отыскать небесные координаты объекта `name` в астрономической базе данных SIMBAD (необходимо подключение к интернету). Теперь координаты не нужно помнить наизусть, ура!

In [ ]:
# проверим, какие у NGC_2808 координаты в SIMBAD
NGC_2808 = SkyCoord.from_name("NGC 2808")
print(NGC_2808)
print(coord.ra - NGC_2808.ra)
print(coord.dec - NGC_2808.dec)

### Использование массивов NumPy с `SkyCoord`

Создадим объект `SkyCoord` для нескольких небесных тел, используя **массивы NumPy**.

Определим:
- Массив **прямых восхождений** (`ra_deg`) в градусах
- Массив **склонений** (`dec_deg`) в градусах  
- Массив **расстояний** (`dist_kpc`) в килопарсеках
- Список **названий** 10 наиболее известных шаровых скоплений.

Передавая эти массивы непосредственно в конструктор `SkyCoord` вместе с подходящими единицами измерения, можно создать структуру из положений этих 10 скоплений. Каждый элемент структуры включает полную информацию о положении скопления в 3-мерном пространстве.

In [ ]:
ra_deg = np.array([6.023625, 13.18875, 78.528625, 81.046667, 138.0108,
                   201.6975, 229.63875, 250.4225, 265.175833, 322.493958])
dec_deg = np.array([-72.081283, -26.582806, -40.046875, -24.524139, -64.8625,
                    -47.479722, 2.081, 36.460278, -53.674694, 12.16625])

cluster_names = np.array([
    "NGC 104",    # 47 Тукана
    "NGC 288",
    "NGC 1851",
    "NGC 1904",   # M79
    "NGC 2808",
    "NGC 5139",   # Омега Центавра
    "NGC 5904",   # M5
    "NGC 6205",   # M13
    "NGC 6397",
    "NGC 7078"    # M15
])

dist_kpc = np.array([
    4.5, 8.9, 12.1, 12.9, 9.6,
    5.2, 7.5, 7.7, 2.3, 10.4
])  #расстояния из GAIA/DR3

glob_cluster_coords = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg, distance = dist_kpc*u.kpc, frame='icrs')
print(glob_cluster_coords)

In [ ]:
# Посчитаем линейное и угловое расстояние между NGC 104 и NGC 7078 с помощью соответствующих функций
sep_3d = glob_cluster_coords[0].separation_3d(glob_cluster_coords[9])
sep_ang = glob_cluster_coords[0].separation(glob_cluster_coords[9])
print(f"Линейное расстояние между {cluster_names[0]} и {cluster_names[9]}: {sep_3d.to(u.kpc):.2f}")
print(f"Угловое расстояние между {cluster_names[0]} и {cluster_names[9]}: {sep_ang.to(u.degree):.2f}")

In [ ]:
# Можно преобразовать это хозяйство в pandas-подобную таблицу с единицами измерения
# о таблицах поговорим подробнее в следующем ноутбуке
table = glob_cluster_coords.to_table()
table['name'] = cluster_names
table

In [ ]:
# Если хочется посмотреть, например, на прямоугольные координаты
glob_cluster_coords.cartesian

## EarthLocation: земные координаты

Координаты места наблюдения можно задать с помощью класса `EarthLocation`. Этот класс позволяет создать места наблюдений, которые затем можно использовать для пересчета локальных координат на определенный момент времени.

Не будем вдаваться в множество деталей, связанных с земными системами отсчета -- будем считать их геоцентрическими и связаными с Международной Земной системой отсчета ITRS, как и делается по-умолчанию в `astropy`.

Создадим объект через геодезические координаты [Коуровского 1.2-метрового телескопа](https://1meter.kourovka.ru/), а также для обсерватории имени Кека:

In [ ]:
from astropy.coordinates import EarthLocation

kourovka_1m = EarthLocation.from_geodetic(lat='+57:02:11N', lon='03h58m10sE', height=290 * u.m) # координаты Коуровского метрового телескопа в WSG84
print(kourovka_1m.get_itrs())

keck = EarthLocation.of_site('Keck Observatory') # список доступных обсерваторий можно посмотреть через EarthLocation.get_site_names()
print(keck.geodetic)

In [ ]:
# Обращаться к положениям можно как к любым другим углам
print(keck.geodetic.lon) # долгота обсерватории Кека
print(kourovka_1m.geocentric[2]) # геоцентрическая z-координата Коуровки, высота над земным экватором

## Время: модуль astropy.time

Для задания определенного момента времени наблюдения можно использовать объект класса `Time`. Задать его можно множеством способов: через всемирное время, юлианскую дату, обычную дату, и т.д.

In [ ]:
from astropy.time import Time

now = Time.now() # текущий момент времени
new_year = Time("2026-01-01", scale='utc') # ближайший НГ
fiftyk = Time("50000", format='mjd', scale='utc') # момент, когда модифицированная юлианская дата MJD=50000

now

In [ ]:
print(f"Текущий момент времени: {now.iso}")
print(f"Текущий юлианский день: {now.jd}")

print(f"Модифицированный юлианский день на новый год: {new_year.mjd}")
print(f"Дата, когда MJD=50000: {fiftyk.iso}")

In [ ]:
# встроенными методами можно, например, посчитать среднее звездное время на нулевом меридиане
now.sidereal_time('mean', 'tio')

## Горизонтальная система координат `AltAz`

Рассмотрим наиболее распространенную задачу — переход от видимых координат в виде высоты (altitude) и азимута (azimuth) в определенном месте в определенное время к небесной системе координат (RA, Dec) и обратно. 

По факту, нужно совершить несколько преобразований:
- Из (h, Az) перейти в ITRS, зная координаты обсерватории и время;
- Из ITRS через промежуточную GIRS перейти к ICRS.

`Astropy` обеспечивает такие переходы между координатами быстро и безболезненно согласно последним стандартам МАС. Так как для работы необходимы параметры ориентации Земли, необходимо подключение к интернету, чтобы `astropy` мог загрузить нужные данные из  [Международной службы вращения Земли (IERS)](https://www.iers.org)

Переход осуществляется с помощью класса `AltAz`, который позволяет это сделать, в том числе с учетом рефракции. По умолчанию, учет рефракции отключен (параметр давления `pressure=0`). Азимуты в этой системе отсчитываются от точки севера N, так что верхняя кульминация светила чаще имеет азимут 180°.

Рассмотрим в качестве примера, какой объект на момент времени `2025-10-28 15:00:00` UTC имеет в Коуровке азимут $75^\circ35^\prime$ и высоту $60^\circ40^\prime$:

In [ ]:
from astropy.coordinates import AltAz, ICRS

t = Time('2025-10-28 15:00:00', scale='utc') # задаем время как объект типа Time
loc = EarthLocation.from_geodetic(lat='+57:02:11N', lon='03h58m10sE', height=290 * u.m) # задаем положение как объект типа EarthLocation

position = AltAz(az='75d35m', alt='60d40m', obstime=t, location=loc) # задаем объект типа AltAz с данными азимутом az, высотой alt
eq_position = position.transform_to(ICRS()) # методом tranform_to перейдем в нужную систему отсчета
eq_position

In [ ]:
# Проверка: проведем обратное преобразование и посмотрим на значение в градусах-минутах-секундах
SkyCoord(eq_position, frame='icrs').transform_to(AltAz(obstime=t, location=loc)).to_string('dms')

В `astropy` также есть функции для подсчета эфемерид тел Солнечной системы. 

Приведем пример такого подсчета для небесных и локальных координат Солнца и Юпитера.

In [ ]:
from astropy.coordinates import get_sun, solar_system_ephemeris, get_body

t = Time('2025-10-28 07:00:00') # полдень по UTC+5 56°50′ с. ш. 60°35′ в. д.
loc = EarthLocation.from_geodetic(lat='+56d50m', lon='60d35m', height=270 * u.m) # центр Екатеринбурга
sun_position = get_sun(t) # положение Солнца в геоцентрической земной системе координат GCRS на момент времени t
sun_position

In [ ]:
sun_position.transform_to(AltAz(obstime=t, location=loc)) # переводим в высоту и азимут

In [ ]:
with solar_system_ephemeris.set('builtin'): # выбираем встроенные и менее точные эфемериды
    jup = get_body('jupiter', t, loc) # положение Солнца в земной системе координат GCRS на момент времени t
jup

In [ ]:
jup.transform_to(AltAz(obstime=t, location=loc))

Подробнее об эфемеридах для тел Солнечной системы можно почитать [здесь](https://docs.astropy.org/en/stable/coordinates/solarsystem.html#astropy-coordinates-solarsystem)

Ссылка на более комплексный пример по преобразованию координат и учету сумерек с помощью `astropy`: <https://docs.astropy.org/en/stable/coordinates/example_gallery_plot_obs_planning.html#sphx-glr-generated-examples-coordinates-plot-obs-planning-py>

## Дополнительные материалы

Мы едва ли коснулись и части того, что может делать `astropy.coordinates`. Для самостоятельного изучения в случае заинтересованности рекомендуется следующая тема:

### Собственные движения и скорости
- Представление движения небесных тел в виде собственных движений и лучевых скоростей
- Пересчет эпохи позиций и скоростей  
[Docs: Motions](https://docs.astropy.org/en/stable/coordinates/motion.html)

# Упражнения

1. Дан список измерений азимута некоторого объекта:
   
   ```
   [0°2′18″, 0°2′31″, 0°4′14″, 359°55′9″, 0°2′8″, 0°1′28″, 359°57′12″, 0°1′38″, 359°58′49″, 0°10′2″]  
   ```
   С помощью класса `Angle` и библиотеки NumPy определите среднее значение и стандартное отклонение этого азимута. Пользуйтесь встроенными функциями и репрезентациями углов. Приведите среднее значение в формате `dms`, а стандартное отклонение - в угловых минутах. *Подсказка:* почитайте про свойство `wrap_angle` подкласса для углов [Longitude](https://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html#astropy.coordinates.Longitude).
2. Создайте список `open_cluster_coords` типа `SkyCoord`  с координатами в ICRS следующих рассеянных скоплений:
   ```
   Pleaiades, Praesepe, Melotte 111, IC 2391, M 41
   ```
   Используйте метод `from_name` для получения их координат из SIMBAD. Какое из этих скоплений отстоит от плоскости Галактики на наибольший угол?
3. Для каждого скопления из массивов `glob_cluster_coords` и `open_cluster_coords` определите созвездие, где оно находится. Для этого воспользуйтесь методом объектов `SkyCoords` [.get_constellation](https://docs.astropy.org/en/stable/api/astropy.coordinates.get_constellation.html#astropy.coordinates.get_constellation). Полученные результаты приведите в DataFrame с тремя колонками: именем скопления `name`, его типом `kind` (шаровое или рассеянное) и созвездием `constellation`.
4. Получите положение Гринвичской обсерватории `g_location` с помощью вызова `EarthLocation.of_site('greenwich')`. Выведите геодезическую широту и долготу этой обсерватории в градусах-минутах-секундах.
5. Создайте объект `birthday_time` типа `Time` для момента вашего дня рождения в 2026 году на местную полночь места вашего рождения (например, если вы родились 10 апреля в Тобольске (UTC+5), то нужно создать `Time('2026-04-09 19:00:00', scale='utc')`).
6. В момент `birthday_time` в месте `g_location` вы собираетесь наблюдать небесное тело с высотой 88° и азимутом 14°. Определите галактическую широту и долготу этого тела.
7. На момент Нового года по UTC+5 `2025-12-31 19:00:00` определите высоту и азимут рассеянных скоплений из списка `open_cluster_coords` в центре Екатеринбурга ($\varphi = +56^\circ50^\circ$, $\lambda=60^\circ35^\prime$).